这里主要用到了`pycox`中的CoxPH模型(DeepSurv)，它是解决连续时间参数情形的模型

In [1]:
#
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pycox
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
import torch
import torchtuples as tt
from pycox.datasets import metabric
from pycox.evaluation import EvalSurv
from pycox.models import CoxPH
np.random.seed(1234)
_ = torch.manual_seed(123)
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 计算weight的函数，输入traning set, calibration set以及一个用来估计P(T=1|X=x)的分类模型
def compute_weight(classification_model,Z_tr,Z_ca):
    '''
    classification_model: 'RF','LR','XGBoost'
    '''
    Z_ca_1 = Z_ca[Z_ca['event']==1]
    X_tr = x_mapper.fit_transform(Z_tr).astype('float32')
    X_ca = x_mapper.transform(Z_ca_1).astype('float32')
    C_tr = Z_tr.iloc[:,-1] # training set的event,用于之后训练分类模型
    # 根据输入选择分类模型
    if classification_model == 'RF':
        from sklearn.ensemble import RandomForestClassifier
        clf = RandomForestClassifier(max_depth=2,random_state=0)
    elif classification_model == 'LR':
        from sklearn.linear_model import LogisticRegression
        clf = LogisticRegression(random_state=0)
    elif classification_model == 'XGBoost':
        import xgboost as xgb
        clf = xgb.XGBClassifier()
    clf.fit(X_tr,C_tr) # 训练分类模型
    p_predict = clf.predict_proba(X_ca)[:,1] # 预测p_hat
    # TBD
    W = np.divide(p_predict,(1-p_predict)) # 估计w_hat
    return (W,clf)

In [43]:
# 计算normalized weight,输入计算的weight，test point，训练过的分类模型
def compute_normalized_weight(W,x,trained_classification_model):
    '''
    x: test point
    '''
    p_predict = trained_classification_model.predict_proba(x)[0,1] # 预测test point对应的T=1的概率
    w_predict = p_predict/(1-p_predict) # 估计p_hat
    normalize_term = np.sum(W)+w_predict 
    p_hat = [i/normalize_term for i in W] # 计算所有病人的p_hat
    p_inf = w_predict/normalize_term # 计算无穷点的weight
    
    return np.array(p_hat+[p_inf])

In [4]:
def generate_distribution(V,p_hat,p_inf):
    V += [np.inf]
    V = np.array(V)
    p_hat = np.array(p_hat+[p_inf])
    return (V,p_hat)

In [45]:
# 计算对应的置信区间，输入nonconformal score, normalized weight p_hat, p_inf,以及指定的percentile
def compute_quantile(V,p_hat,Z_tr,percentile,x,t,model,bh,non_zero_idx):
    '''
    t: 指定的值
    V: 这是calibration set对应的V
    '''
    ch = model.predict_cumulative_hazards(x)
    exp_g_x = ch.loc[non_zero_idx]/bh
    exp_g_x_r = compute_nonconformal_score_single(model,Z_tr,x,t,x_mapper,bh,non_zero_idx)
    if exp_g_x_r is None:
        return -1 # 注意这里需要修改
    V_x = np.log(exp_g_x)-np.log(np.sum(exp_g_x_r))
    p_hat_leave = p_hat[V<=V_x[0]]
    return sum(p_hat_leave)

In [41]:
def weighted_conformal_prediction(V,W,x,t_h,Z_tr,model,trained_clf,percentile,bh,non_zero_idx,epsilon=0.01):
    p_hat = compute_normalized_weight(W,x,trained_clf)
    quantile = 0
    t_l = 0
    while (abs(quantile-percentile)>epsilon) or (quantile<percentile):
        t = (t_l+t_h)/2
        quantile = compute_quantile(V,p_hat,Z_tr,percentile,x,t,model,bh,non_zero_idx)
        if (quantile > percentile) and (abs(quantile-percentile)>epsilon):
            t_h = t
            t = (t_h+t_l)/2
        else:
            t_l = t
            t = (t_h+t_l)/2
    return t

In [7]:
# 划分数据，输入原始数据，选择划分的比例，输出训练集验证集和calibration set
def split_data(df,train_frac=0.6,calibration_frac=0.2):
    tr_df = df.sample(frac=train_frac)
    df = df.drop(tr_df.index)
    ca_df = df.sample(frac=calibration_frac/(1-train_frac))
    df = df.drop(ca_df.index)
    val_df = df
    return (tr_df,val_df,ca_df)

In [8]:
# 读取数据并划分
df_train = metabric.read_df()
Z_tr,Z_val,Z_ca = split_data(df_train)

In [57]:
df_train

,x0,x1,x2,x3,x4,x5,x6,x7,x8,duration,event
0,5.603834,7.811392,10.797988,5.967607,1.0,1.0,0.0,1.0,56.840000,99.333336,0
1,5.284882,9.581043,10.204620,5.664970,1.0,0.0,0.0,1.0,85.940002,95.733330,1
2,5.920251,6.776564,12.431715,5.873857,0.0,1.0,0.0,1.0,48.439999,140.233337,0
3,6.654017,5.341846,8.646379,5.655888,0.0,0.0,0.0,0.0,66.910004,239.300003,0
4,5.456747,5.339741,10.555724,6.008429,1.0,0.0,0.0,1.0,67.849998,56.933334,1
...,...,...,...,...,...,...,...,...,...,...,...
1899,5.946987,5.370492,12.345780,5.741395,1.0,1.0,0.0,1.0,76.839996,87.233330,1
1900,5.339228,5.408853,12.176101,5.693043,1.0,1.0,0.0,1.0,63.090000,157.533340,0
1901,5.901610,5.272237,14.200950,6.139390,0.0,0.0,0.0,1.0,57.770000,37.866665,1
1902,6.818109,5.372744,11.652624,6.077852,1.0,0.0,0.0,1.0,58.889999,198.433334,0


In [9]:
# 将数据标准化
cols_standardize = ['x0', 'x1', 'x2', 'x3', 'x8']
cols_leave = ['x4', 'x5', 'x6', 'x7']

standardize = [([col], StandardScaler()) for col in cols_standardize]
leave = [(col, None) for col in cols_leave]

x_mapper = DataFrameMapper(standardize + leave)

In [10]:
x_train = x_mapper.fit_transform(Z_tr).astype('float32')
x_val = x_mapper.transform(Z_val).astype('float32')
x_ca = x_mapper.transform(Z_ca).astype('float32')

In [11]:
get_target = lambda df: (df['duration'].values, df['event'].values)
y_train = get_target(Z_tr)
y_val = get_target(Z_val)
durations_test, events_test = get_target(Z_ca)
val = x_val, y_val

In [12]:
# 搭建神经网络
in_features = x_train.shape[1]
num_nodes = [32, 32]
out_features = 1
batch_norm = True
batch_size=256
dropout = 0.1
output_bias = False

In [13]:
net = torch.nn.Sequential(
    torch.nn.Linear(in_features, 32),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(32),
    torch.nn.Dropout(0.1),
    
    torch.nn.Linear(32, 32),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(32),
    torch.nn.Dropout(0.1),
    
    torch.nn.Linear(32, out_features)
)

In [14]:
# CoxPH模型，使用Adam优化器
model = CoxPH(net, torch.optim.Adam)

In [15]:
epochs = 512
callbacks = [tt.callbacks.EarlyStopping()]
verbose = True

In [16]:
%%time
log = model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose,
                val_data=val, val_batch_size=batch_size)

0:	[0s / 0s],		train_loss: 4.9349,	val_loss: 4.4688
1:	[0s / 0s],		train_loss: 4.7703,	val_loss: 4.4529
2:	[0s / 0s],		train_loss: 4.6512,	val_loss: 4.4320
3:	[0s / 0s],		train_loss: 4.6480,	val_loss: 4.4125
4:	[0s / 0s],		train_loss: 4.6010,	val_loss: 4.4008
5:	[0s / 0s],		train_loss: 4.5825,	val_loss: 4.3956
6:	[0s / 0s],		train_loss: 4.5567,	val_loss: 4.3962
7:	[0s / 0s],		train_loss: 4.5724,	val_loss: 4.4010
8:	[0s / 0s],		train_loss: 4.5904,	val_loss: 4.4040
9:	[0s / 0s],		train_loss: 4.5713,	val_loss: 4.4020
10:	[0s / 0s],		train_loss: 4.5739,	val_loss: 4.3984
11:	[0s / 0s],		train_loss: 4.5518,	val_loss: 4.3959
12:	[0s / 0s],		train_loss: 4.5459,	val_loss: 4.3967
13:	[0s / 0s],		train_loss: 4.5631,	val_loss: 4.3964
14:	[0s / 1s],		train_loss: 4.5520,	val_loss: 4.3953
15:	[0s / 1s],		train_loss: 4.5310,	val_loss: 4.3949
16:	[0s / 1s],		train_loss: 4.5671,	val_loss: 4.3932
17:	[0s / 1s],		train_loss: 4.5202,	val_loss: 4.3856
18:	[0s / 1s],		train_loss: 4.5173,	val_loss: 4.3810
19:

In [17]:
baseline_hazards = model.compute_baseline_hazards()
non_zero_idx = baseline_hazards[baseline_hazards>0].index[1] # 计算第一个非零元素的索引
bh = baseline_hazards.loc[non_zero_idx]

In [19]:
def compute_nonconformal_score_single(model,Z_tr,x,t,x_mapper,bh,non_zero_idx):
    R = Z_tr[Z_tr['duration']>=t] # 找到at risk的人的covariates
    if len(R) == 0: # 如果没找到at risk的人就跳过
        return None
    x_R = x_mapper.transform(R).astype('float32')
    ch_r = model.predict_cumulative_hazards(x_R)
    exp_g_r = ch_r.loc[non_zero_idx]/bh
    return exp_g_r

In [52]:
# 计算nonconformal score的函数，给定一个预测hazard的模型，training set
# 和calibration set以及base hazard，输出结果
def compute_nonconformal_score(model,x_mapper,Z_tr,Z_ca,bh,non_zero_idx):
    '''
    model: 预测模型
    Z_tr: traning set
    Z_ca: calibration set
    return: calibration set的 nonconformal score
    '''
    Z_ca_1 = Z_ca[Z_ca['event']==1] # calibration set中发病的样本
    x_ca = x_mapper.transform(Z_ca_1).astype('float32')
    durations_test_1, events_test_1 = get_target(Z_ca_1)
    cumulative_hazards = model.predict_cumulative_hazards(x_ca)
    exp_g = cumulative_hazards.loc[non_zero_idx].div(bh)
    V = list()
    for i in range(len(x_ca)): # nonconformal score
        exp_g_r = compute_nonconformal_score_single(model,Z_tr,x_ca[i],durations_test_1[i],x_mapper,bh,non_zero_idx)
        if exp_g_r is None:
            V.append(np.inf)
        else:
            V.append(np.log(exp_g[i])-np.log(np.sum(exp_g_r)))
    return np.array(V+[np.inf])

In [21]:
Z_ca_1 = Z_ca.sample(frac=0.99)
Z_ca_2 = Z_ca.drop(Z_ca_1.index)

In [27]:
# 计算calibration set中Delta = 1的每个数据的权重
W,clf = compute_weight('XGBoost',Z_tr,Z_ca_1)

In [28]:
baseline_hazards = model.compute_baseline_hazards()
non_zero_idx = baseline_hazards[baseline_hazards>0].index[1] # 计算第一个非零元素的索引
bh = baseline_hazards.loc[non_zero_idx]

In [36]:
V = compute_nonconformal_score(model,x_mapper,Z_tr,Z_ca_1,bh,non_zero_idx)

In [50]:
t_h = np.max(df_train['duration'])

355.20001220703125

In [51]:
x_ca_2 = x_mapper.transform(Z_ca_2).astype('float32')
for i in range(len(x_ca_2)):
    x = np.array([x_ca_2[i]])
    print(weighted_conformal_prediction(V,W,x,t_h,Z_tr,model,clf,0.95,bh,non_zero_idx))

255.3000087738037
260.8500089645386
244.20000839233398
263.625009059906


In [ ]:
#
X_idx = Z_ca[Z_ca['event']==1].to_numpy() # 找到Delta_i = 1的数据，转化成numpy数据
Z_ca_1 = Z_ca[Z_ca['event']==1] # 找到Delta_i = 1的数据
Z_ca_test = Z_ca_1.sample(frac=0.01) # 将calibration set中划出一部分用来test
Z_ca_1 = Z_ca_1.drop(Z_ca_test.index)
x_test_1 = x_mapper.transform(Z_ca_1).astype('float32')
durations_test_1, events_test_1 = get_target(Z_ca_1)
cumulative_hazards = model.predict_cumulative_hazards(x_test_1)
exp_g = cumulative_hazards.loc[non_zero_idx].div(bh)
# 计算nonconformal score
V = list()
ch = model.predict_cumulative_hazards(x_test_1) 
exp_g = ch.iloc[2,:]/bh
p_hat, p_inf= list(),list()
for i in range(len(x_test_1)):
    t = durations_test_1[i]
    R = Z_tr[Z_tr['event']==0][Z_tr['duration']>=t] # 找到at risk的人的covariates
    if len(R) == 0: # 如果没找到at risk的人就跳过
        continue
    x_R = x_mapper.fit_transform(R).astype('float32')
    ch_r = model.predict_cumulative_hazards(x_R)
    exp_g_r = ch_r.loc[non_zero_idx]/bh
    V.append(np.log(exp_g[i])-np.log(np.sum(exp_g_r)))

In [56]:
x_ca_2

array([[-0.91468555,  1.1301656 , -0.47822767, -0.789998  ,  0.7613344 ,
         1.        ,  1.        ,  0.        ,  1.        ],
       [-0.32096985,  0.7625667 ,  0.01102955, -0.5552923 , -0.8143126 ,
         1.        ,  0.        ,  1.        ,  1.        ],
       [ 2.2719808 , -1.0495    , -0.80151224,  1.5298384 ,  0.299441  ,
         0.        ,  1.        ,  0.        ,  0.        ],
       [-0.5856291 ,  1.038398  , -0.6645023 , -0.30214155,  0.46566126,
         0.        ,  1.        ,  0.        ,  1.        ]],
      dtype=float32)

In [55]:
for i in range(len(x_ca_test)):
    print('Test point %d: %f' % (i,weighted_conformal_prediction(V,W,x_ca_test[i][np.newaxis,:],clf,0.95)[1]))

NameError: name 'x_ca_test' is not defined

In [53]:
x_ca_test

NameError: name 'x_ca_test' is not defined